In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")


import datajoint_utils as du
from importlib import reload

INFO - 2021-03-18 22:34:43,605 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-18 22:34:43,607 - settings - Setting database.user to celiib
INFO - 2021-03-18 22:34:43,608 - settings - Setting database.password to newceliipass
INFO - 2021-03-18 22:34:43,620 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-18 22:34:43,622 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-18 22:34:43,637 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-18 22:34:43,985 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-18 22:34:43,989 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-18 22:34:43,990 - settings - Setting database.user to celiib
INFO - 2021-03-18 22:34:43,991 - settings - Setting database.password to newceliipass
INFO - 2021-03-18 22:34:43,992 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-18 22:34:43,992 - settings - Setting database.user to celiib
INFO - 2021-03-18 22:34:43,993 - settings - Setting database.password to newceliipass
INFO - 2021-03-18 22:34:43,995 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-18 22:34:44,378 - settings - Setting enable_python_native_blobs to True


In [2]:
test_mode = True

In [3]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-18 22:34:44,454 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-18 22:34:44,455 - settings - Setting database.user to celiib
INFO - 2021-03-18 22:34:44,456 - settings - Setting database.password to newceliipass
INFO - 2021-03-18 22:34:44,457 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-18 22:34:44,458 - settings - Setting database.user to celiib
INFO - 2021-03-18 22:34:44,459 - settings - Setting database.password to newceliipass
INFO - 2021-03-18 22:34:44,461 - settings - Setting enable_python_native_blobs to True


Sleeping 32 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-03-18 22:34:44,881 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [4]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [5]:
import meshlab
meshlab.set_meshlab_port(current_port=None)
temporary_folder = 'decimation_temp'
meshlab_scripts = {}

No port chosen so picked random port 9558


In [6]:
#so that it will have the adapter defined
from datajoint_utils import *

In [7]:
# key_source = minnie.Mesh.proj() * (minnie.DecimationConfig & 'decimation_ratio=0.25') & (minnie.AllenProofreading() & dict(month=3,day=18,year=2021)).proj()
# minnie.Decimation() & key_source

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691135081756919,0,0.25,2990029,6102497,=BLOB=
864691135337764326,0,0.25,1657003,3339842,=BLOB=
864691135340990917,0,0.25,1028903,2083646,=BLOB=
864691135341029829,0,0.25,1408282,2835919,=BLOB=
864691135345208735,0,0.25,1755370,3532920,=BLOB=
864691135345280671,0,0.25,2631717,5309289,=BLOB=
864691135373423816,0,0.25,3383405,6837747,=BLOB=
864691135382504922,0,0.25,1778136,3607784,=BLOB=
864691135382719450,0,0.25,2268160,4584441,=BLOB=
864691135403793646,0,0.25,1272236,2608554,=BLOB=


In [10]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

from minfig.minnie65_config import external_decimated_mesh_path

@schema
class Decimation(dj.Computed):
#     definition = minnie.Decimation.describe(printout=False)
    key_source = minnie.Mesh.proj() * (minnie.DecimationConfig & 'decimation_ratio=0.25') & (minnie.AllenProofreading() & dict(month=3,day=18,year=2021)).proj()

    # Creates hf file at the proper location, returns the filepath of the newly created file
    @classmethod
    def make_file(cls, segment_id, version, decimation_ratio, vertices, faces):
        """Creates hf file at the proper location, returns the filepath of the newly created file"""

        assert vertices.ndim == 2 and vertices.shape[1] == 3
        assert faces.ndim == 2 and faces.shape[1] == 3

        filename = f'{segment_id}_{version}_{int(decimation_ratio*100):02}.h5'
        filepath = os.path.join(external_decimated_mesh_path, filename)
        with h5py.File(filepath, 'w') as hf:
            hf.create_dataset('segment_id', data=segment_id)
            hf.create_dataset('version', data=version)
            hf.create_dataset('decimation_ratio', data=float(decimation_ratio))
            hf.create_dataset('vertices', data=vertices)
            hf.create_dataset('faces', data=faces)

        return filepath

    @classmethod
    def make_entry(cls, segment_id, version, decimation_ratio, vertices, faces):
        key = dict(
            segment_id=segment_id,
            version=version,
            decimation_ratio=decimation_ratio,
            n_vertices=len(vertices),
            n_faces=len(faces)
        )

        filepath = cls.make_file(segment_id, version, decimation_ratio, vertices, faces)

        cls.insert1(dict(key, mesh=filepath), allow_direct_insert=True)

    

    def make(self, key):
        print(key)
        mesh = (minnie.Mesh & key).fetch1('mesh')
        segment_id = key['segment_id']
        version = key['version']
        decimation_ratio = key['decimation_ratio']
        print(f"Mesh size: n_vertices = {len(mesh.vertices)}, n_faces = {len(mesh.faces)}")

        if decimation_ratio not in meshlab_scripts:
            meshlab_scripts[decimation_ratio] = meshlab.Decimator(decimation_ratio, temporary_folder, overwrite=False)
        mls_func = meshlab_scripts[decimation_ratio]

        try:
            expected_filepath = os.path.join(external_decimated_mesh_path, f'{segment_id}_{version}.h5')
            if not os.path.isfile(expected_filepath):
                new_mesh, _path = mls_func(mesh.vertices, mesh.faces, segment_id)
                new_vertices, new_faces = new_mesh.vertices, new_mesh.faces

                self.make_entry(
                    segment_id=segment_id,
                    version=version,
                    decimation_ratio=decimation_ratio,
                    vertices=new_vertices,
                    faces=new_faces,
                    )
            else:
                print('File already exists.')
                with h5py.File(expected_filepath, 'r') as hf:
                    vertices = hf['vertices'][()].astype(np.float64)
                    faces = hf['faces'][()].reshape(-1, 3).astype(np.uint32)
                self.insert1(dict(key, n_vertices=len(vertices), n_faces=len(faces), mesh=expected_filepath), allow_direct_insert=True)
        except Exception as e:
            minnie.DecimationError.insert1(dict(key, log=str(e)))
            print(e)
            raise e
    

# Running the Populate

In [12]:
curr_table = (minnie.schema.jobs & "table_name='__decimation'")
#curr_table.delete()
#curr_table.delete()
#curr_table.delete()

In [13]:
import time
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decimation.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decimation.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for Decimation populate = {time.time() - start_time}")

INFO - 2021-03-18 22:35:53,186 - autopopulate - Found 6 keys to populate
INFO - 2021-03-18 22:35:53,196 - connection - Transaction started
INFO - 2021-03-18 22:35:53,198 - autopopulate - Populating: {'segment_id': 864691135132762784, 'version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started
{'segment_id': 864691135132762784, 'version': 0, 'decimation_ratio': Decimal('0.25')}
Mesh size: n_vertices = 3924249, n_faces = 7826112
xvfb-run -n 9558 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135132762784.off -o /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135132762784_decimated.off -s /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/decimation_meshlab_25800615.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135132762784.off
removed temporary output file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135132762784_decimated.off


INFO - 2021-03-18 22:40:04,425 - connection - Transaction committed and closed.
INFO - 2021-03-18 22:40:04,431 - connection - Transaction started
INFO - 2021-03-18 22:40:04,433 - autopopulate - Populating: {'segment_id': 864691135462553757, 'version': 0, 'decimation_ratio': Decimal('0.25')}


{'segment_id': 864691135462553757, 'version': 0, 'decimation_ratio': Decimal('0.25')}
Mesh size: n_vertices = 5373896, n_faces = 10720724
xvfb-run -n 9558 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135462553757.off -o /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135462553757_decimated.off -s /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/decimation_meshlab_25800615.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135462553757.off
removed temporary output file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135462553757_decimated.off


INFO - 2021-03-18 22:46:07,391 - connection - Transaction committed and closed.
INFO - 2021-03-18 22:46:07,398 - connection - Transaction started
INFO - 2021-03-18 22:46:07,401 - autopopulate - Populating: {'segment_id': 864691135590161419, 'version': 0, 'decimation_ratio': Decimal('0.25')}


...

{'segment_id': 864691135724472235, 'version': 0, 'decimation_ratio': Decimal('0.25')}
Mesh size: n_vertices = 5350036, n_faces = 10674521
xvfb-run -n 9558 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135724472235.off -o /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135724472235_decimated.off -s /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/decimation_meshlab_25800615.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135724472235.off
removed temporary output file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691135724472235_decimated.off


INFO - 2021-03-18 23:09:30,577 - connection - Transaction committed and closed.
INFO - 2021-03-18 23:09:30,601 - connection - Transaction started
INFO - 2021-03-18 23:09:30,603 - autopopulate - Populating: {'segment_id': 864691136286740931, 'version': 0, 'decimation_ratio': Decimal('0.25')}


{'segment_id': 864691136286740931, 'version': 0, 'decimation_ratio': Decimal('0.25')}
Mesh size: n_vertices = 2467636, n_faces = 4930980
xvfb-run -n 9558 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691136286740931.off -o /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691136286740931_decimated.off -s /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/decimation_meshlab_25800615.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691136286740931.off
removed temporary output file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/decimation_temp/neuron_864691136286740931_decimated.off


INFO - 2021-03-18 23:12:05,763 - connection - Transaction committed and closed.


Populate Done
Total time for Decimation populate = 2172.5972883701324
